In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sc.constants as cs
from sc import payoff as po
from yq.scripts import simulation as yqsm

print(pd.__version__)

In [ ]:
paths_arr = yqsm.read_sim_data('gbm/archive', '20231111_195045_022812', pd.Timestamp('2023-08-14'), pd.Timestamp('2023-08-15'))
print(paths_arr)

In [ ]:
#prep for multipath

df_sim_array_14 = []
for df in paths_arr[0]:
    df_sim_array_14.append(df.rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'}))

df_sim_array_15 = []
for df in paths_arr[1]:
    df_sim_array_15.append(df.rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'}))

In [ ]:
prices_14 = po.pricing_multiple(df_sim_array_14)
print(sum(prices_14) / len(prices_14))

prices_15 = po.pricing_multiple(df_sim_array_15)
print(prices_15)
print(sum(prices_15) / len(prices_15))

In [ ]:
# from pathlib import Path
# from yq.utils import path as yq_path
# import os
# paths_arr = yqsm.read_sim_data('gbm', '20231111_195045_022812', pd.Timestamp('2023-08-09'), pd.Timestamp('2023-08-15'))
# df_sim = paths_arr[0][0]

# fig, ax = plt.subplots(figsize=(10,6))

# hist_data = po.get_historical_assets_all()
# hist_df = hist_data[(hist_data.index >= cs.INITIAL_FIXING_DATE) 
#                            & (hist_data.index <= cs.FINAL_FIXING_DATE)]
# for asset in cs.ASSET_NAMES:
#     ax.plot(hist_df.index, hist_df[asset], alpha=0.5, label=asset)
# for col in df_sim.columns:
#     ax.plot(df_sim.index, df_sim[col], alpha=0.5, label=col)


# title_str = f"PPD: "
# plt.title(title_str)
# plt.legend(loc='upper right')
# plt.tight_layout()
# stor_dir = yq_path.get_plots_path(Path(os.getcwd()).parent)                     
# stor_dir.mkdir(parents=True, exist_ok=True)
# file_path = stor_dir.joinpath(f'test.png')
# plt.savefig(file_path, bbox_inches='tight')

In [ ]:
df_plot = df_sim_array_15[2].copy(deep = True)
print(po.payouts(df_plot, barrierHit = False))
for asset in cs.ASSET_NAMES:
    df_plot[asset] = df_plot[asset] / cs.INITIAL_LEVELS[asset]
plt.figure(figsize = (15, 8))
plt.plot(df_plot)
plt.axhline(y=1, c = "black", alpha = 0.5)
plt.axhline(y = 0.6, c = "red", alpha = 0.5)
plt.axvline(x = cs.FINAL_FIXING_DATE, c = "darkgreen")
for date in cs.EARLY_REDEMPTION_OBSERVATION_DATES:
    plt.axvline(x = date, c = "green", alpha = 0.5)



In [ ]:
#Entire process

first_date = pd.Timestamp('2023-08-09')
last_date = pd.Timestamp('2023-11-09')

#load simulated paths from csvs
total_data = yqsm.read_sim_data('gbm/archive', '20231111_195045_022812', first_date, last_date)

est_price = []
for single_day in total_data:
    #rename columns
    df_sim_array = []
    for df in single_day:
        df_sim_array.append(df.rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'}))
    
    prices = po.pricing_multiple(df_sim_array)
    est_price.append(sum(prices) / len(prices))
print(est_price)




In [ ]:
df_product = po.get_product_price(pd.Timestamp('2023-11-09'))
df_product = df_product.loc[first_date : last_date]
df_product['Est_Price'] = est_price

plt.plot(df_product)